# RadTract example pipeline

This notebook will show you how to use RadTract to calculate radiomics tractometry features. The used data can be found in [resources/example/](https://github.com/MIC-DKFZ/radtract/tree/main/resources/example/) in this repository. Make sure to install RadTract before running this notebook: `pip install radtract`.

## Minimal example

To run RadTract on a single subject, you need at least the following items:

1. A tractogram, i.e., a collection of streamlines, for the tract you are interested in (here `resources/example/CST_right.trk`). 
2. The image you want to calculate features from, for example a fractional anisotropy (FA) map (here `resources/example/fa.nii.gz`).

The code below is the minimal version of how to use these items to calculate your tract-specific features. 

<span style="color:red;">**Keep in mind to read the longer example below before using RadTract for a multi-subject study. This is particularly important to keep the number of parcels and the tract orientations constant across subject!**</span>

In [ ]:
# parcellate tract
!radtract_parcellate --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz --output resources/example/CST_right_parcellation.nii.gz

# calculate features using this parcellation
!radtract_features --parcellation resources/example/CST_right_parcellation.nii.gz --map resources/example/fa.nii.gz --output resources/example/CST_right_features.csv

## Longer example

In the following sections, the individual steps to perform a RadtractAnalyis are described in more detail. To obtain a full list of options for a command, just execute it in the command line withou any options. 

### How to get the tracts/streamlines and start-region-images in the first place

You can obtain tractograms/streamlines of individual tracts with various software tools, e.g. MRtrix or MITK Diffusion. Since it is easy to use and fully automatic, we recommend TractSeg (https://github.com/MIC-DKFZ/TractSeg/) for larger studies. Simply install TractSeg via `pip install tractseg` and make sure pytorch is also installed (`pip install torch`).

In [ ]:
# segment 72 tracts in the input dwi
!TractSeg -i resources/example/dwi.nii.gz --raw_diffusion_input --keep_intermediate_files

# segment start- and end-regions of all tracts. these are required for the following tractography as well as for the parcellation
!TractSeg -i resources/example/tractseg_output/peaks.nii.gz --output_type endings_segmentation -o resources/example/tractseg_output/ --keep_intermediate_files

# create tract orientation maps (TOM) for tractography 
!TractSeg -i resources/example/tractseg_output/peaks.nii.gz --output_type TOM -o resources/example/tractseg_output/ --keep_intermediate_files

# run tractography (here only of the corpus callosum). we use more streamlines (10000) than the defaul to obtain a better tract coverage.
!Tracking -i resources/example/tractseg_output/peaks.nii.gz --tracking_format trk --nr_fibers 10000 -o resources/example/tractseg_output/ --bundles CC

### Tract preprocessing/filtering

While the above commands already yield smooth tracts between the segmented start- and end-regions, in some cases it is necessary to further process and filter the resulting tracts to remove outliers, to remove prematurely ending streamlines, etc. RadTract provides the following commands to do this:

1. `radtract_filter_curvature`: Remove streamlines with a high curvature over a certain distance.
2. `radtract_filter_length`: Remove short or long streamlines.
3. `radtract_filter_density`: Remove streamlines with a certain fraction of streamline points located in sparsely populated tract regions, which are likely outliers.
4. `radtract_filter_visitationcount`: Same as above, but using the visitation count instead of the density, i.e. the accumulated length of all streamline segments in a voxel.
5. `radtract_filter_maskoverlap`: Remove streamlines with a certain fraction of streamline points located outside of the provided mask.
6. `radtract_filter_endpoints`: Enables various endpoint filtering methods, e.g. keep stremlines with exactly one endpoint in provided mask or with two endpoints located in different label regions etc.

3 and 4 can either be used with precalculated density/visitation count maps or with a dummy image as refernce, that is used for calculating the maps internally.


### Parcellation

As mentioned above, it is very important to keep the number of parcels and the orientation of tracts constant when parcellating tracts for multiple subjects. 

#### The number of parcels

To fix the number of parcels when running `radtract_parcellate`, this parameter can be explicitely specified using `--num_parcels`. In this case the number of parcels is not estimated automatically per tract and subject, but the provided number is used. To obtain a suitable value for this parameter, the command `radtract_estimate_num_parcels` on a set of tracts, e.g. the left CST of all your subjects. The command will ouput a CSV file with the estimated number of parcels for this tract for each subject. For parcellating your tracts, the median or rounded mean of these values is probably a good choice.


#### Unify orientation of tracts across subjects

Tractograms do not contain information about where a tract starts and where it ends. For example in case of the CST, the start of the tract could be near the cortex or near the spine, and if this changes between subjects, the labels of the parcellation will be reversed and no menaingful analysis can be performed. To avoid this, you can pass an ROI image to RadTract that defines where the tract starts, using `--start`. Start regions can be segmented manually or calculated automaticall, e.g., using TractSeg.

#### Parcellation types

RadTract offers multiple types (`--type`) of parcellations, namely hyperplane (default), centerline and static. Furthermore, the parcellation can happen in voxel-space (default, required for radiomics analysis) or in streamline-space (classic tractometry, option `--streamline_space`). In the figure below, the different types of parcellations are illustrated. In the next cell you will find the corresponding commands used to create these results.

| Hyperplane-based parcellation | Hyperplane-based parcellation | Static parcellation |
|:---:|:---:|:---:|
| <img src="resources/hyperplane_streamline_parcellation.png" height="400"></br>Streamline space | <img src="resources/centerline_streamline_parcellation.png" height="400"></br>Streamline space | <img src="resources/static_streamline_parcellation.png" height="400"></br>Streamline space |
| <img src="resources/hyperplane_voxel_parcellation.png" height="500"> </br>Maximum intensity projection in voxel space | <img src="resources/centerline_voxel_parcellation.png" height="500"> </br>Maximum intensity projection in voxel space | |


In [ ]:
!radtract_parcellate --type hyperplane --output resources/example/CST_left_parcellation_hyperplane.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

!radtract_parcellate --type centerline --output resources/example/CST_left_parcellation_centerline.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

!radtract_parcellate --type hyperplane --streamline_space --output resources/example/CST_left_streamline-parcellation_hyperplane.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

!radtract_parcellate --type centerline --streamline_space --output resources/example/CST_left_streamline-parcellation_centerline.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

!radtract_parcellate --type static --output resources/example/CST_left_streamline-parcellation_static.nii.gz --num_parcels 15 --save_intermediate_files --streamlines resources/example/CST_right.trk --reference resources/example/fa.nii.gz

### Feature calculation

The actual feature calculation is pretty straight forward and the tl;dr example above works perfectly well for FA maps. For other maps, e.g., ADC, it might be neccessary to adjust the pyradiomics parameter file, particularly the parameter `binWidt`. Check out the available .yaml parameter files in [radtract/](https://github.com/MIC-DKFZ/radtract/tree/main/radtract). For details on pyradiomics, please check out the pyradiomics [documentation](https://pyradiomics.readthedocs.io/en/latest/).